In [ ]:
!pip install numpy librosa matplotlib scikit-learn tqdm

In [ ]:
from google.colab import files
import zipfile
import os

import numpy as np
import librosa
import librosa.feature
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif

from sklearn.ensemble import IsolationForest

import IPython.display as ipd
import random

import joblib

In [ ]:
zip_path = "data.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted files:", os.listdir(extract_path))

Extracted files: ['non_drone', 'drone']


In [ ]:
drone_path = os.path.join(extract_path, "drone")
non_drone_path = os.path.join(extract_path, "non_drone")

# Function to extract MFCC features
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs, axis=1)

X, y = [], []

for file in tqdm(os.listdir(drone_path), desc="Processing Drone Audio"):
    if file.endswith(".wav"):
        X.append(extract_features(os.path.join(drone_path, file)))
        y.append("drone")

for file in tqdm(os.listdir(non_drone_path), desc="Processing Non-Drone Audio"):
    if file.endswith(".wav"):
        X.append(extract_features(os.path.join(non_drone_path, file)))
        y.append("non-drone")

X_ = np.array(X)
y_ = np.array(y)
print(f"Feature shape: {X_.shape}, Labels shape: {y_.shape}")

Processing Non-Drone Audio: 100%|██████████| 1728/1728 [00:30<00:00, 56.95it/s]

Feature shape: (3060, 13), Labels shape: (3060,)


In [ ]:
import os
import numpy as np
import librosa
import librosa.feature
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier

def load_dataset(drone_path, non_drone_path):
    X, y = [], []

    for file in os.listdir(drone_path):
        if file.endswith(".wav"):
            X.append(extract_features(os.path.join(drone_path, file)))
            y.append("drone")

    for file in os.listdir(non_drone_path):
        if file.endswith(".wav"):
            X.append(extract_features(os.path.join(non_drone_path, file)))
            y.append("non-drone")

    return np.array(X), np.array(y)

def select_features(X, y):
    # Remove low-variance features
    selector = VarianceThreshold(threshold=0.01)
    X_filtered = selector.fit_transform(X)

    # Select top features using ANOVA F-score
    selector = SelectKBest(score_func=f_classif, k=20)
    X_filtered = selector.fit_transform(X_filtered, y)

    # Feature importance using Random Forest
    model = RandomForestClassifier()
    model.fit(X_filtered, y)
    importances = model.feature_importances_
    top_features = np.argsort(importances)[-20:]  # Select top 20 features

    return X_filtered[:, top_features]

def preprocess_audio_data(drone_path, non_drone_path):
    X, y = load_dataset(drone_path, non_drone_path)
    X_selected = select_features(X, y)
    print(f"Final feature shape: {X_selected.shape}, Labels shape: {y.shape}")
    return X_selected, y

In [ ]:
X_selected, y = preprocess_audio_data(drone_path, non_drone_path)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=20 is greater than n_features=13. All the features will be returned.
  warnings.warn(


Final feature shape: (3060, 13), Labels shape: (3060,)


In [ ]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_selected, y_encoded, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Binary Classification Accuracy: {accuracy:.2f}")

Binary Classification Accuracy: 0.98


In [ ]:
# Save the trained IsolationForest model
joblib.dump(clf, "clf_model.pkl")

# Download the model file
from google.colab import files
files.download("clf_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
drone_files = [os.path.join(drone_path, f) for f in os.listdir(drone_path) if f.endswith(".wav")]
non_drone_files = [os.path.join(non_drone_path, f) for f in os.listdir(non_drone_path) if f.endswith(".wav")]

print(f"Drone samples: {len(drone_files)}, Non-drone samples: {len(non_drone_files)}")

X_drone_only = np.array([extract_features(file) for file in tqdm(drone_files, desc="Processing Drone Files for Anomaly Detection")])

anomaly_model = IsolationForest(contamination=0.1, random_state=42)
anomaly_model.fit(X_drone_only)

X_all = np.array([extract_features(file) for file in tqdm(drone_files + non_drone_files, desc="Testing Anomaly Detection")])

predictions = anomaly_model.predict(X_all)
predictions = ["drone" if p == 1 else "non-drone" for p in predictions]

print("Anomaly detection results:", predictions[:10])

Drone samples: 1332, Non-drone samples: 1728


Processing Drone Files for Anomaly Detection:   0%|          | 0/1332 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Testing Anomaly Detection:   0%|          | 0/3060 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Testing Anomaly Detection: 100%|██████████| 3060/3060 [02:15<00:00, 22.55it/s]

Anomaly detection results: ['non-drone', 'drone', 'drone', 'drone', 'drone', 'drone', 'drone', 'drone', 'drone', 'drone']


In [ ]:
num_samples = 5
random_indices = random.sample(range(len(X_all)), num_samples)

print("Playing samples with predicted labels:")
for idx in random_indices:
    file_path = (drone_files + non_drone_files)[idx]
    predicted_label = predictions[idx]

    print(f"Predicted: {predicted_label} | File: {file_path}")

    ipd.display(ipd.Audio(file_path, rate=22050))


Playing samples with predicted labels:
Predicted: non-drone | File: /content/dataset/non_drone/1-17092-B-271.wav


Predicted: non-drone | File: /content/dataset/non_drone/1-20545-A-281.wav


Predicted: non-drone | File: /content/dataset/non_drone/1-47250-A-412.wav


Predicted: drone | File: /content/dataset/drone/mixed_24-bebop_002_.wav


Predicted: non-drone | File: /content/dataset/non_drone/3-154439-A-172.wav


In [ ]:
# ground truth labels (1 = drone, -1 = non-drone)
true_labels = [1] * len(drone_files) + [-1] * len(non_drone_files)

# predictions ("drone"/"non-drone") to numerical labels
predicted_labels = [1 if p == "drone" else -1 for p in predictions]

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Anomaly Detection Accuracy: {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=["Non-Drone", "Drone"]))

print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

Anomaly Detection Accuracy: 0.89

Classification Report:
              precision    recall  f1-score   support

   Non-Drone       0.92      0.89      0.91      1728
       Drone       0.86      0.90      0.88      1332

    accuracy                           0.89      3060
   macro avg       0.89      0.90      0.89      3060
weighted avg       0.90      0.89      0.89      3060


Confusion Matrix:
[[1539  189]
 [ 134 1198]]


In [ ]:
# !pip install sounddevice scipy librosa

In [ ]:
# # Save the trained IsolationForest model
# joblib.dump(anomaly_model, "drone_anomaly_model.pkl")

# # Download the model file
# from google.colab import files
# files.download("drone_anomaly_model.pkl")